In [1]:
import xgboost as xgb
import shap
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from shap_analysis import SHAPAnalyzer, function_map
import optuna

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [ ]:
df = pd.read_csv("thesis_cleaned_transformed_histone_dataset_categorical(H3K23me1).tsv", sep="\t")

X = df.drop(columns = ['Gene Expression (FPKM)_log'])
y = df['Gene Expression (FPKM)_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def evaluate_model(X_train, X_test, y_train, y_test, params=None, optuna_study=None):
    """
    Evaluate the model either using the provided parameters or the Optuna study's best trial.
    
    Parameters:
    - X_train (pd.DataFrame): Training feature data.
    - X_test (pd.DataFrame): Test feature data.
    - y_train (pd.Series): Training target data.
    - y_test (pd.Series): Test target data.
    - params (dict): Parameters for the XGBoost model. If None, use `optuna_study`.
    - optuna_study (optuna.Study): Optuna study object containing the best trial.
    
    Returns:
    - model: Trained XGBoost model.
    - total_score (float): The evaluation score based on SHAP analysis.
    - result_summary (dict): Summary of matches and mismatches.
    - results_df (pd.DataFrame): Detailed SHAP results for features.
    """
    # Determine parameters: Use the best trial from Optuna if no parameters are provided
    if optuna_study is not None and params is None:
        params = optuna_study.best_trial.params
        params.update({
            'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda'
        })

    # Prepare the data
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Train the model
    model = xgb.train(
        params, 
        dtrain, 
        num_boost_round=100, 
        evals=[(dtest, 'validation')],
        early_stopping_rounds=10, 
        verbose_eval=False
    )

    # SHAP analysis
    background_indices = np.random.choice(X_train.shape[0], size=200, replace=False)
    background_sample = X_train.iloc[background_indices]

    sample_indices = np.random.choice(X_test.shape[0], size=200, replace=False)
    sample = X_test.iloc[sample_indices]

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)

    # Use the SHAPAnalyzer to calculate metrics
    analyzer = SHAPAnalyzer(X_test, shap_values, function_map)
    analyzer.calculate_high_value_shap_means()

    results_df, total_score, result_summary = analyzer.get_results()

    print(f"Results Summary: {result_summary}")

    return model, total_score, result_summary, results_df


In [5]:
model, total_score, result_summary, resdults_df = evaluate_model(X_train, X_test, y_train, y_test)
print(total_score)
print(result_summary)

[18:17:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 9, 'Mismatch': 7, 'No Evaluation': 1}
0.6137400286097545
{'Match': 9, 'Mismatch': 7, 'No Evaluation': 1}


In [6]:
def feature_selection(X_train, X_test, y_train, y_test):
    remaining_features = X_train.columns.tolist()
    selected_features = []

    while remaining_features:
        print(f"Remaining Features: {remaining_features}")
        reduced_X_train = X_train[remaining_features]
        reduced_X_test = X_test[remaining_features]

        model, total_score, result_summary, results_df = evaluate_model(
            reduced_X_train, reduced_X_test, y_train, y_test
            )

        mismatch_features = results_df[results_df["Result"] == "Mismatch"]
        
        if mismatch_features.empty:
            selected_features = remaining_features
            break

        mismatch_to_remove = mismatch_features.sort_values(
            by = "Mean SHAP Value (High)", ascending = False
        ).iloc[0]['Features']

        print(f"Removing mismatch feature: {mismatch_to_remove}")

        remaining_features.remove(mismatch_to_remove)

    return selected_features

In [7]:
def feature_selection_with_optuna(X_train, X_test, y_train, y_test, function_map):
    remaining_features = X_train.columns.tolist()
    selected_features = []

    def optuna_objective(trial, X_train, X_test, y_train, y_test):
        params = {
            'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda',
            'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.05),
            'max_depth': trial.suggest_int("max_depth", 10, 30),
            'subsample': trial.suggest_float("subsample", 0.4, 1.0),
            'gamma': trial.suggest_float("gamma", 0.0, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
            'lambda': trial.suggest_float("lambda", 1e-3, 15),
            'alpha': trial.suggest_float("alpha", 1e-3, 15),
    }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_test, label=y_test)

        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dvalid, 'validation')],
                          early_stopping_rounds=10,
                          verbose_eval=False)

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)

        analyzer = SHAPAnalyzer(X_test, shap_values, function_map)
        analyzer.calculate_high_value_shap_means()

        results_df, total_score, result_summary = analyzer.get_results()
        total_mismatches = result_summary.get("Mismatch", 0)
        total_matches = result_summary.get("Match", 0)
        accuracy = total_matches / (total_matches + total_mismatches) if (total_matches + total_mismatches) > 0 else 0

        # Define the objective score
        alpha = 1.0  # Weight for accuracy
        beta = 1.0   # Weight for mismatch count
        objective_score = alpha * accuracy
        
        trial.set_user_attr("result_summary", result_summary)
        trial.set_user_attr("results_df", results_df)
        
        return objective_score

    while remaining_features:
        print(f"Remaining Features: {remaining_features}")
        
        # Run Optuna for the current subset of features
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: optuna_objective(trial, 
                                                      X_train[remaining_features], 
                                                      X_test[remaining_features], 
                                                      y_train, y_test), 
                       n_trials=40)
        
        best_params = study.best_params
        print(f"Best Parameters: {best_params}")

        # Evaluate model with the current subset
        reduced_X_train = X_train[remaining_features]
        reduced_X_test = X_test[remaining_features]

        model, total_score, result_summary, results_df = evaluate_model(
            reduced_X_train, reduced_X_test, y_train, y_test, optuna_study=study
)

        mismatch_features = results_df[results_df["Result"] == "Mismatch"]

        if mismatch_features.empty:
            selected_features = remaining_features
            break

        # Sort mismatching features and remove the highest impact one
        mismatch_to_remove = mismatch_features.sort_values(
            by="Mean SHAP Value (High)", ascending=False
        ).iloc[0]["Features"]

        print(f"Removing mismatch feature: {mismatch_to_remove}")
        remaining_features.remove(mismatch_to_remove)

    return selected_features, study


In [8]:
def feature_selection_remove_all_mismatches(X_train, X_test, y_train, y_test):
    remaining_features = X_train.columns.tolist()
    selected_features = []

    while True:
        print(f"Remaining features: {remaining_features}")
        
        # Reduce the dataset to remaining features
        reduced_X_train = X_train[remaining_features]
        reduced_X_test = X_test[remaining_features]

        # Train the model and perform SHAP analysis
        model, total_score, result_summary, results_df = evaluate_model(
            reduced_X_train, reduced_X_test, y_train, y_test
        )

        # Identify mismatched features
        mismatch_features = results_df[results_df["Result"] == "Mismatch"]["Features"].tolist()
        
        if not mismatch_features:  # No mismatches left
            selected_features = remaining_features
            break

        print(f"Removing mismatched features: {mismatch_features}")

        # Remove all mismatched features
        remaining_features = [feature for feature in remaining_features if feature not in mismatch_features]

    return selected_features


In [9]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

total_matches = 0
total_mismatches = 0

features = function_map["Features"]
known_functions = function_map["Known Function"]

feature_scores = {}

for i, feature_to_leave_out in enumerate(features):
    print(f"\n--- Performing LOO for feature {feature_to_leave_out} ---\n")

    reduced_X_train = X_train.drop(columns = [feature_to_leave_out])
    reduced_X_test = X_test.drop(columns = [feature_to_leave_out])

    selected_features, study = feature_selection_with_optuna(reduced_X_train, reduced_X_test, y_train, y_test, function_map)

    final_X_train = X_train[selected_features + [feature_to_leave_out]]
    final_X_test = X_test[selected_features + [feature_to_leave_out]]

    print(f"\n--- Evaluating model with feature {feature_to_leave_out} added back ---\n")

    model, total_score, result_summary, results_df = evaluate_model(
        final_X_train, final_X_test, y_train, y_test, optuna_study=study
        )

    known_function = known_functions[i]
    feature_result = results_df[results_df["Features"] == feature_to_leave_out]

    if not feature_result.empty:
        match_result = feature_result["Result"].values[0]
        print(f"Feature: {feature_to_leave_out}, Known Function: {known_function}, Result: {match_result}")

        if match_result == "Match":
            total_matches += 1
        else:
            total_mismatches += 1

        feature_scores[feature_to_leave_out] = {
            "Total Score": total_score,
            "Known Function": known_function,
            "Result": match_result
        }

total_features = total_matches + total_mismatches
accuracy = total_matches / total_features if total_features > 0 else 0

print("\n--- Feature Analysis Results ---")

for feature, score in feature_scores.items():
    print(f"Feature: {feature}, Total Score: {score['Total Score']}, Known Function: {score['Known Function']}, Result: {score['Result']}")

print("\n--- Overall Accuracy ---")
print(f"Total Matches: {total_matches}")
print(f"Total Mismatches: {total_mismatches}")
print(f"Accuracy: {accuracy:.2f}")


--- Performing LOO for feature H3K4me1 ---

Remaining Features: ['H2A.2', 'H2A.13', 'H3K4me3', 'H3K36me3', 'H4K5Ac', 'H2A.Z.11', 'expression_category', 'H3K27me3_log', 'H3K9Ac_log', 'cpg_percentage_log', 'H3K9K14Ac_log', 'H3Ac_log', 'chh_percentage_log', 'H4K20me1_log', 'chg_percentage_log', 'H2A.W.7_log', 'H3K9me1_log', 'H2A.W.6_log', 'H2A.Z.9_log', 'H3K23me1_log', 'H3K9me2_log']


[18:17:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[18:17:32] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[18:17:39] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[18:17:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using 

Best Parameters: {'learning_rate': 0.013193333635654558, 'max_depth': 28, 'subsample': 0.7866046408546474, 'gamma': 0.7244040171307976, 'colsample_bytree': 0.7094159048005868, 'lambda': 1.3408855949018241, 'alpha': 1.3259860683055658}


[18:24:57] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14, 'Mismatch': 1, 'No Evaluation': 1}
Removing mismatch feature: H4K5Ac
Remaining Features: ['H2A.2', 'H2A.13', 'H3K4me3', 'H3K36me3', 'H2A.Z.11', 'expression_category', 'H3K27me3_log', 'H3K9Ac_log', 'cpg_percentage_log', 'H3K9K14Ac_log', 'H3Ac_log', 'chh_percentage_log', 'H4K20me1_log', 'chg_percentage_log', 'H2A.W.7_log', 'H3K9me1_log', 'H2A.W.6_log', 'H2A.Z.9_log', 'H3K23me1_log', 'H3K9me2_log']


...

Best Parameters: {'learning_rate': 0.004006929054713544, 'max_depth': 12, 'subsample': 0.9070544386431157, 'gamma': 0.284668975235714, 'colsample_bytree': 0.8425872249755906, 'lambda': 9.942318123335548, 'alpha': 4.5872018009898685}


[03:55:23] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14}

--- Evaluating model with feature H3K23me1_log added back ---



[03:55:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14, 'No Evaluation': 1}
Feature: H3K23me1_log, Known Function: Unknown, Result: No Evaluation

--- Feature Analysis Results ---
Feature: H3K4me1, Total Score: 0.9504962702048942, Known Function: Activating, Result: Mismatch
Feature: H3K9me2_log, Total Score: 0.25640461000148207, Known Function: Repressive, Result: Match
Feature: H3K4me3, Total Score: 1.2852750404272228, Known Function: Activating, Result: Match
Feature: H3K36me3, Total Score: 0.2637489909720898, Known Function: Activating, Result: Match
Feature: H4K5Ac, Total Score: 1.0305808932462242, Known Function: Activating, Result: Mismatch
Feature: H3K27me3_log, Total Score: 0.4979779444402084, Known Function: Repressive, Result: Match
Feature: H3K9Ac_log, Total Score: 0.9557464671379421, Known Function: Activating, Result: Match
Feature: cpg_percentage_log, Total Score: 0.7295486981747672, Known Function: Repressive, Result: Match
Feature: chh_percentage_log, Total Score: 1.5150523303309456, Known Fun

## Accuracy as a metric

In [10]:
#No Feature Selection
optuna.logging.set_verbosity(optuna.logging.INFO)

total_matches = 0
total_mismatches = 0

features = function_map["Features"]
known_functions = function_map["Known Function"]

feature_scores = {}

for i, feature_to_leave_out in enumerate(features):
    print(f"\n--- Performing LOO for feature {feature_to_leave_out} ---\n")

    #Features for optuna optimization
    reduced_X_train = X_train.drop(columns = [feature_to_leave_out])
    reduced_X_test = X_test.drop(columns = [feature_to_leave_out])

    #Optuna Optimization
    def optuna_objective(trial, reduced_X_train, reduced_X_test, y_train, y_test):
        params = {
            'booster': 'gbtree',
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'device': 'cuda',
            'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.05),
            'max_depth': trial.suggest_int("max_depth", 10, 30),
            'subsample': trial.suggest_float("subsample", 0.4, 1.0),
            'gamma': trial.suggest_float("gamma", 0.0, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
            'lambda': trial.suggest_float("lambda", 1e-3, 15),
            'alpha': trial.suggest_float("alpha", 1e-3, 15),
            }

        dtrain = xgb.DMatrix(reduced_X_train, label=y_train)
        dvalid = xgb.DMatrix(reduced_X_test, label=y_test)

        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dvalid, 'validation')],
                          early_stopping_rounds=10,
                          verbose_eval=False)

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(reduced_X_test)

        analyzer = SHAPAnalyzer(reduced_X_test, shap_values, function_map)
        analyzer.calculate_high_value_shap_means()

        results_df, total_score, result_summary = analyzer.get_results()
        total_mismatches = result_summary.get("Mismatch", 0)
        total_matches = result_summary.get("Match", 0)
        accuracy = total_matches / (total_matches + total_mismatches) if (total_matches + total_mismatches) > 0 else 0

        # Define the objective score
        alpha = 1.0  # Weight for accuracy
        beta = 1.0   # Weight for mismatch count
        objective_score = alpha * accuracy
        
        trial.set_user_attr("result_summary", result_summary)
        trial.set_user_attr("results_df", results_df)
        
        return objective_score
    
    # Create a study object and optimize
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: optuna_objective(trial, 
                                                    reduced_X_train, 
                                                    reduced_X_test, 
                                                    y_train, y_test), 
                    n_trials=40)

    # final_X_train = X_train
    # final_X_test = X_test

    print(f"\n--- Evaluating model with feature {feature_to_leave_out} added back ---\n")

    model, total_score, result_summary, results_df = evaluate_model(
        X_train, X_test, y_train, y_test, optuna_study=study
        )

    known_function = known_functions[i]
    feature_result = results_df[results_df["Features"] == feature_to_leave_out]

    if not feature_result.empty:
        match_result = feature_result["Result"].values[0]
        print(f"Feature: {feature_to_leave_out}, Known Function: {known_function}, Result: {match_result}")

        if match_result == "Match":
            total_matches += 1
        else:
            total_mismatches += 1

        feature_scores[feature_to_leave_out] = {
            "Total Score": total_score,
            "Known Function": known_function,
            "Result": match_result
        }

total_features = total_matches + total_mismatches
accuracy = total_matches / total_features if total_features > 0 else 0

print("\n--- Feature Analysis Results ---")

for feature, score in feature_scores.items():
    print(f"Feature: {feature}, Total Score: {score['Total Score']}, Known Function: {score['Known Function']}, Result: {score['Result']}")

print("\n--- Overall Accuracy ---")
print(f"Total Matches: {total_matches}")
print(f"Total Mismatches: {total_mismatches}")
print(f"Accuracy: {accuracy:.2f}")

[I 2025-03-13 03:55:27,841] A new study created in memory with name: no-name-5f28b0df-9e3b-4281-b39a-ab20bd304afc



--- Performing LOO for feature H3K4me1 ---



[03:55:30] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-13 03:55:32,227] Trial 0 finished with value: 0.8 and parameters: {'learning_rate': 0.0176062152167535, 'max_depth': 13, 'subsample': 0.6238892554975046, 'gamma': 0.12369703872956894, 'colsample_bytree': 0.9974056698378566, 'lambda': 4.4712578598390245, 'alpha': 3.242548504160648}. Best is trial 0 with value: 0.8.
[03:55:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-13 03:55:35,895] Trial 1 finished with value: 0.8666666666666667 and parameters: {'learning_rate': 0.039478376291899986,


--- Evaluating model with feature H3K4me1 added back ---



[03:58:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-13 03:58:33,788] A new study created in memory with name: no-name-67d0a4a9-1bf8-4bb9-bba4-ce9b765d8c12


...

Results Summary: {'Match': 13, 'Mismatch': 3, 'No Evaluation': 1}
Feature: H3K9K14Ac_log, Known Function: Activating, Result: Match

--- Performing LOO for feature H3K23me1_log ---



[05:07:01] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-13 05:07:02,708] Trial 0 finished with value: 0.8125 and parameters: {'learning_rate': 0.017442171455456238, 'max_depth': 21, 'subsample': 0.5616273806561785, 'gamma': 0.48014708926641025, 'colsample_bytree': 0.8790536736076778, 'lambda': 12.652378191379597, 'alpha': 5.760391101658183}. Best is trial 0 with value: 0.8125.
[05:07:05] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
[I 2025-03-13 05:07:06,827] Trial 1 finished with value: 0.6875 and parameters: {'learning_rate': 0.04568654770233403, 'max


--- Evaluating model with feature H3K23me1_log added back ---



[05:11:31] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1707159369174/work/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Results Summary: {'Match': 14, 'Mismatch': 2, 'No Evaluation': 1}
Feature: H3K23me1_log, Known Function: Unknown, Result: No Evaluation

--- Feature Analysis Results ---
Feature: H3K4me1, Total Score: 0.7987461101729423, Known Function: Activating, Result: Mismatch
Feature: H3K9me2_log, Total Score: 1.1884491329547018, Known Function: Repressive, Result: Match
Feature: H3K4me3, Total Score: 1.059755397029221, Known Function: Activating, Result: Match
Feature: H3K36me3, Total Score: 1.1901368650142103, Known Function: Activating, Result: Match
Feature: H4K5Ac, Total Score: 0.946912654035259, Known Function: Activating, Result: Mismatch
Feature: H3K27me3_log, Total Score: 0.9107447473797947, Known Function: Repressive, Result: Match
Feature: H3K9Ac_log, Total Score: 1.0977196507155895, Known Function: Activating, Result: Match
Feature: cpg_percentage_log, Total Score: 1.0289717165287584, Known Function: Repressive, Result: Match
Feature: chh_percentage_log, Total Score: 1.347812474705278

## Score as a metric